In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import tensorflow as tf

# Ensure we can import from the local folder
try:
    from aesindy.solvers import SynthData
    from aesindy.training import TrainModel
except ImportError:
    raise ImportError("Run this script from the root directory of the project (where 'aesindy' folder is located).")

In [ ]:
CASE_NAME = 'lorenz' 

# ==========================================
# 2. EXPERIMENT CONFIGURATIONS
# derived from aesindy/dynamical_models.py & testcases/
# ==========================================
configs = {
    'lorenz': {
        'model': 'lorenz',
        'system_coefficients': [10, 28, 8/3],  # sigma, rho, beta
        'normalization': [1/40, 1/40, 1/40],   # Scale to approx [-1, 1]
        'latent_dim': 3,                       # 3D chaotic attractor
        'poly_order': 2,                       # Quadratic terms (xy, xz)
        'include_sine': False,
        'input_dim': 128,                      # Time-delay window size
        'dt': 0.01,                            # Time step
        'tend': 20,                            # Simulation duration
        'n_ics': 100,                          # Number of training trajectories
    },
    'rossler': {
        'model': 'rossler',
        'system_coefficients': [0.2, 0.2, 5.7], # a, b, c
        'normalization': [1/40, 1/40, 1/40],
        'latent_dim': 3,
        'poly_order': 2,
        'include_sine': False,
        'input_dim': 128,
        'dt': 0.04,                            # Rossler is slower, bigger dt
        'tend': 50,
        'n_ics': 100,
    },
    'predator_prey': {
        'model': 'predator_prey',
        'system_coefficients': [1.0, 0.1, 1.5, 0.75], # alpha, beta, gamma, delta
        'normalization': [1/10, 1/10],
        'latent_dim': 2,                       # 2D limit cycle
        'poly_order': 2,
        'include_sine': False,
        'input_dim': 128,
        'dt': 0.01,
        'tend': 20,
        'n_ics': 100,
    },
    'pendulum': {
        'model': 'pendulum',
        'system_coefficients': [9.8, 1],       # g, L
        'normalization': [1/5, 1/5],
        'latent_dim': 2,
        'poly_order': 1,                       # Linear terms...
        'include_sine': True,                  # ...plus Sine terms (sin(x))
        'input_dim': 128,
        'dt': 0.01,
        'tend': 10,
        'n_ics': 100,
    }
}

In [ ]:
q = 5

In [5]:
# ==========================================
# 3. GLOBAL HYPERPARAMETERS
# Used across all experiments (from testcases/default_params.py)
# ==========================================
selected_config = configs[CASE_NAME]
params = {}
params.update(selected_config)

In [6]:
from testcases.default_params import params as default_params

In [7]:
params = default_params.copy()
params['model'] = 'lorenz'

In [ ]:
### Parameters for the methods 

# Training & Network
params['case'] = CASE_NAME
params['widths_ratios'] = [0.5, 0.25]   # Encoder: 128 -> 64 -> 32 -> latent
params['activation'] = 'elu'
params['use_bias'] = True
params['batch_size'] = 32
params['max_epochs'] = 1000             # Paper often used 500-2000
params['learning_rate'] = 1e-3
params['learning_rate_sched'] = False
params['print_progress'] = True
params['print_frequency'] = 50          # Print equations every 50 epochs

# SINDy (Equation Discovery)
params['coefficient_threshold'] = 0.05  # Threshold for sparsity
params['threshold_frequency'] = 50      # Pruning frequency
params['coefficient_initialization'] = 'constant'
params['fix_coefs'] = False             # Learn coefficients
params['exact_features'] = False        # Discover structure automatically
params['sindy_pert'] = 0.0
params['ode_net'] = False               # Use SINDy, not NeuralODE

# Loss Function Weights (Critical for convergence)
params['loss_weight_rec'] = 1.0         # Reconstruction
params['loss_weight_sindy_z'] = 0.05    # Derivatives in Latent space
params['loss_weight_sindy_x'] = 1e-4    # Derivatives in Input space
params['loss_weight_sindy_regularization'] = 1e-5 # Sparsity (L1)
params['loss_weight_integral'] = 0.0    # Optional integral loss
params['loss_weight_x0'] = 0.0
params['loss_weight_layer_l2'] = 0.0
params['loss_weight_layer_l1'] = 0.0

# Misc
params['noise'] = 0.0
params['data_path'] = 'results'
params['save_checkpoints'] = False
params['train_ratio'] = 0.8
params['svd_dim'] = None
params['scale'] = False
params['sparse_weighting'] = None
params['trainable_auto'] = True
params['use_sindycall'] = False
params['ode_net_widths'] = [] # Unused

params['fixed_coefficient_mask'] = False  # Important: Set to False to strictly "discover" equations. Otherwise, the problem is just a Parameter Tuning Optimization problem 

In [8]:
print(f"--- Running Experiment: {CASE_NAME.upper()} ---")
print(f"System: {params['model']}")
print(f"Coefficients: {params['system_coefficients']}")

# 1. Generate Data
print("\nGenerating Synthetic Data...")
S = SynthData(model=params['model'],
            args=params['system_coefficients'],
            noise=params['noise'],
            input_dim=params['input_dim'],
            normalization=params['normalization'])

# Run simulation to create the Hankel matrix (time-delay embeddings)
S.run_sim(params['n_ics'], params['tend'], params['dt'])

--- Running Experiment: LORENZ ---
System: lorenz
Coefficients: None

Generating Synthetic Data...
generating solutions..


100%|██████████| 5/5 [00:00<00:00,  6.32it/s]


In [9]:
params['max_epochs'] = 5
params['latent_dim'] = 3

In [10]:
# 2. Train Model
print("\nStarting Training...")
trainer = TrainModel(S, params)
# trainer.fit()


Starting Training...


In [11]:
# Check if SINDy variables exist and are trainable
sindy_vars = trainer.model.sindy.trainable_variables

print(f"Number of SINDy trainable variables: {len(sindy_vars)}")
if len(sindy_vars) > 0:
    print(f"First variable name: {sindy_vars[0].name}")
    print(f"First variable shape: {sindy_vars[0].shape}")
else:
    print("⚠️ WARNING: The SINDy layer has NO trainable variables!")

Number of SINDy trainable variables: 0
⚠️ WARNING: The SINDy layer has NO trainable variables!


In [ ]:
# 3. Output Results
print("\n" + "="*30)
print("FINAL DISCOVERED EQUATIONS")
print("="*30)

# Get coefficients
coeffs = trainer.model.sindy.coefficients.numpy()

# Define feature names based on poly_order
feature_names = ["1"]
latent_dim = params['latent_dim']

# Linear terms
for i in range(latent_dim):
    feature_names.append(f"z{i}")

# Quadratic terms (if poly_order >= 2)
if params['poly_order'] >= 2:
    for i in range(latent_dim):
        for j in range(i, latent_dim):
            feature_names.append(f"z{i}z{j}")
            
# Sine terms (if include_sine is True)
if params['include_sine']:
    for i in range(latent_dim):
        feature_names.append(f"sin(z{i})")

# Print in readable format
print(f"{'Feature':<10} | {'d(z0)/dt':<10} | {'d(z1)/dt':<10} | {'d(z2)/dt' if latent_dim>2 else ''}")
print("-" * 45)
for i, name in enumerate(feature_names):
    row = coeffs[i]
    # Only print if row has non-zero values (approx)
    if np.any(np.abs(row) > 1e-4):
        val0 = f"{row[0]:.4f}"
        val1 = f"{row[1]:.4f}"
        val2 = f"{row[2]:.4f}" if latent_dim > 2 else ""
        print(f"{name:<10} | {val0:<10} | {val1:<10} | {val2}")


In [ ]:
# 3. Output Results
print("\n" + "="*30)
print("FINAL DISCOVERED EQUATIONS")
print("="*30)

# Get coefficients
coeffs = trainer.model.sindy.coefficients.numpy()

# Define feature names based on poly_order
feature_names = ["1"]
latent_dim = params['latent_dim']

# Linear terms
for i in range(latent_dim):
    feature_names.append(f"z{i}")

# Quadratic terms (if poly_order >= 2)
if params['poly_order'] >= 2:
    for i in range(latent_dim):
        for j in range(i, latent_dim):
            feature_names.append(f"z{i}z{j}")
            
# Sine terms (if include_sine is True)
if params['include_sine']:
    for i in range(latent_dim):
        feature_names.append(f"sin(z{i})")

# Print in readable format
print(f"{'Feature':<10} | {'d(z0)/dt':<10} | {'d(z1)/dt':<10} | {'d(z2)/dt' if latent_dim>2 else ''}")
print("-" * 45)
for i, name in enumerate(feature_names):
    row = coeffs[i]
    # Only print if row has non-zero values (approx)
    if np.any(np.abs(row) > 1e-4):
        val0 = f"{row[0]:.4f}"
        val1 = f"{row[1]:.4f}"
        val2 = f"{row[2]:.4f}" if latent_dim > 2 else ""
        print(f"{name:<10} | {val0:<10} | {val1:<10} | {val2}")
